In [338]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
from openpyxl import *
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from collections import Counter

# Web scraping

In [339]:
url_link='https://www.imdb.com/chart/top'
url=url_link
url

'https://www.imdb.com/chart/top'

In [340]:
df = pd.read_html(url, header=0)

In [341]:
df=pd.DataFrame(df[0])
df.drop(columns=['Unnamed: 0','Unnamed: 4','Your Rating'],inplace=True)
df

,Rank & Title,IMDb Rating
0,1. The Shawshank Redemption (1994),9.2
1,2. The Godfather (1972),9.2
2,3. The Dark Knight (2008),9.0
3,4. The Godfather Part II (1974),9.0
4,5. 12 Angry Men (1957),9.0
...,...,...
245,246. Dersu Uzala (1975),8.0
246,247. The Help (2011),8.0
247,248. Aladdin (1992),8.0
248,249. Gandhi (1982),8.0


In [342]:
df_sorted = df.sort_values(by='IMDb Rating',ascending=False).head(10)
df_sorted

,Rank & Title,IMDb Rating
0,1. The Shawshank Redemption (1994),9.2
1,2. The Godfather (1972),9.2
2,3. The Dark Knight (2008),9.0
3,4. The Godfather Part II (1974),9.0
4,5. 12 Angry Men (1957),9.0
5,6. Schindler's List (1993),8.9
6,7. The Lord of the Rings: The Return of the K...,8.9
7,8. Pulp Fiction (1994),8.8
8,9. The Lord of the Rings: The Fellowship of t...,8.8
9,"10. Il buono, il brutto, il cattivo (1966)",8.8


In [343]:
fig =px.bar(df_sorted,x='Rank & Title',y='IMDb Rating',color='Rank & Title',
           labels={'Rank & Title':'الاسم', 'IMDb Rating':' تقييم'},
           title='اعلى عشر افلام')
fig.show()

In [344]:
df_sorted = df.sort_values(by='IMDb Rating',ascending=True).head(10)
df_sorted

,Rank & Title,IMDb Rating
249,250. Dances with Wolves (1990),8.0
225,226. The Exorcist (1973),8.0
224,225. The Best Years of Our Lives (1946),8.0
223,224. Groundhog Day (1993),8.0
222,223. Pather Panchali (1955),8.0
221,222. The Wizard of Oz (1939),8.0
220,221. Before Sunset (2004),8.0
219,220. Into the Wild (2007),8.0
218,219. Network (1976),8.0
226,227. The Incredibles (2004),8.0


In [345]:
fig =px.bar(df_sorted,x='Rank & Title',y='IMDb Rating',color='Rank & Title',
           labels={'Rank & Title':'الاسم', 'IMDb Rating':' تقييم'},
           title='اقل عشر افلام')
fig.show()

# Scrape Twitter

In [ ]:
# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(
    'كأس العالم للأندية until:2023-02-15 since:2023-01-01').get_items()):
    if i>10000:
        break
    attributes_container.append([tweet.user.username,tweet.user.displayname , tweet.date, 
                                 tweet.likeCount, tweet.sourceLabel, tweet.content, 
                                 tweet.lang,tweet.coordinates, tweet.place])
     
# Creating a dataframe to load the list
tweets_df = pd.DataFrame(attributes_container, columns=["User", "Name", "Date Created", 
                                                        "Number of Likes", "Source of Tweet", "Tweet", 
                                                        "Language", "Coordinates","place"])

tweets_df["cleanTweet"] = tweets_df["Tweet"].apply(lambda s: re.sub(r'(?:\@\w+)','', s))# remove string like: @kmkk 
tweets_df["cleanTweet"] = tweets_df["cleanTweet"].apply(lambda s: re.sub(r'(?:[a-zA-Z]+)','', s))# remove english word
tweets_df["cleanTweet"] = tweets_df["cleanTweet"].apply(lambda s: re.sub(  r'([$@&!~#^*+=-_]+)'    ,'', s)) 
tweets_df = tweets_df[tweets_df['Language'] ==  "ar"]
tweets_df['Date Created'] = tweets_df['Date Created'].apply(lambda a: pd.to_datetime(a).date())
print(tweets_df)
tweets_df.to_excel('TwitterData3.xlsx')

In [347]:
df = pd.read_excel('TwitterData3.xlsx')
df.drop(columns='Unnamed: 0',inplace=True)
df

,User,Name,Date Created,Number of Likes,Source of Tweet,Tweet,Language,Coordinates,place,cleanTweet
0,AX98jfg8k94,SAMIII,2023-02-14,0,Twitter for iPhone,@iwamzyan 😂😂😂 كأس العالم للاندية هل تطن ان الم...,ar,NaN,NaN,😂😂😂 كأس العالم للاندية هل تطن ان المغرب قام ب...
1,hebasay39744927,🦅♥️🇪🇬 heba.sayed‏,2023-02-14,38,Twitter for Android,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...,ar,NaN,NaN,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...
2,2oios,-ﺎلسسَآطي ‏𖤐,2023-02-14,0,Twitter for iPhone,@toldo2002 @tooothy_ وبإذن الله ان التاسعة تتح...,ar,NaN,NaN,وبإذن الله ان التاسعة تتحق السنه ذي وكاس الع...
3,mohamed90108304,Mohamed Metwally Shaarawi ( ابو بـودئ )🇪🇬♥️🇸🇦,2023-02-14,1,Twitter for iPhone,"أعلن الاتحاد الدولي لكرة القدم ""#فيفا"" تطبيق ن...",ar,NaN,NaN,"أعلن الاتحاد الدولي لكرة القدم ""فيفا"" تطبيق نظ..."
4,fstp1396,الزكرتي,2023-02-14,2,Twitter for iPhone,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...,ar,NaN,NaN,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...
...,...,...,...,...,...,...,...,...,...,...
9819,aljeban,أحمد | AHMED,2023-02-12,1,Twitter for iPhone,#ريال_مدريد بطل #كأس_العالم_للأندية 2022\n ...,ar,NaN,NaN,ريالمدريد بطل كأسالعالمللأندية 2022\n ...
9820,sayyidaty,برنامج سيدتي,2023-02-12,3,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...
9821,YaHalaShow,برنامج ياهلا,2023-02-12,1,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...
9822,sa3etshabab,ساعة شباب,2023-02-12,0,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...


In [348]:
df.columns

Index(['User', 'Name', 'Date Created', 'Number of Likes', 'Source of Tweet',
       'Tweet', 'Language', 'Coordinates', 'place', 'cleanTweet'],
      dtype='object')

In [349]:
df.columns=['User', 'Name', 'Date_Created', 'Number_of_Likes', 'Source_of_Tweet', 'Tweet',
            'Language', 'Coordinates','place','cleanTweet']

In [350]:
df

,User,Name,Date_Created,Number_of_Likes,Source_of_Tweet,Tweet,Language,Coordinates,place,cleanTweet
0,AX98jfg8k94,SAMIII,2023-02-14,0,Twitter for iPhone,@iwamzyan 😂😂😂 كأس العالم للاندية هل تطن ان الم...,ar,NaN,NaN,😂😂😂 كأس العالم للاندية هل تطن ان المغرب قام ب...
1,hebasay39744927,🦅♥️🇪🇬 heba.sayed‏,2023-02-14,38,Twitter for Android,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...,ar,NaN,NaN,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...
2,2oios,-ﺎلسسَآطي ‏𖤐,2023-02-14,0,Twitter for iPhone,@toldo2002 @tooothy_ وبإذن الله ان التاسعة تتح...,ar,NaN,NaN,وبإذن الله ان التاسعة تتحق السنه ذي وكاس الع...
3,mohamed90108304,Mohamed Metwally Shaarawi ( ابو بـودئ )🇪🇬♥️🇸🇦,2023-02-14,1,Twitter for iPhone,"أعلن الاتحاد الدولي لكرة القدم ""#فيفا"" تطبيق ن...",ar,NaN,NaN,"أعلن الاتحاد الدولي لكرة القدم ""فيفا"" تطبيق نظ..."
4,fstp1396,الزكرتي,2023-02-14,2,Twitter for iPhone,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...,ar,NaN,NaN,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...
...,...,...,...,...,...,...,...,...,...,...
9819,aljeban,أحمد | AHMED,2023-02-12,1,Twitter for iPhone,#ريال_مدريد بطل #كأس_العالم_للأندية 2022\n ...,ar,NaN,NaN,ريالمدريد بطل كأسالعالمللأندية 2022\n ...
9820,sayyidaty,برنامج سيدتي,2023-02-12,3,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...
9821,YaHalaShow,برنامج ياهلا,2023-02-12,1,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...
9822,sa3etshabab,ساعة شباب,2023-02-12,0,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...


In [351]:
df_sorted = df.sort_values(by='Number_of_Likes',ascending=False).head(10)
fig =px.bar(df_sorted,x='User',y='Number_of_Likes',hover_data=['User', 'Name','cleanTweet'],color='User',
           labels={'User':'المستخدم', 'Number_of_Likes':'عدد الاعجاب'},
           title='اعلى عشر تغريدات حازت على اعجاب الاخرين')
fig.show()

In [352]:
df_sorted

,User,Name,Date_Created,Number_of_Likes,Source_of_Tweet,Tweet,Language,Coordinates,place,cleanTweet
4764,saudiFF,الاتحاد السعودي لكرة القدم,2023-02-14,5239,Twitter Web App,الاتحاد الدولي لكرة القدم FIFA:\n\nالسعودية تس...,ar,NaN,NaN,الاتحاد الدولي لكرة القدم :\n\nالسعودية تستضيف...
4089,AbdulazizTF,عبدالعزيز بن تركي الفيصل,2023-02-14,4609,Twitter for iPhone,امتدادًا للنجاحات في تنظيم البطولات والأحداث ا...,ar,NaN,NaN,امتدادًا للنجاحات في تنظيم البطولات والأحداث ا...
4759,SaudiNews50,أخبار السعودية,2023-02-14,3134,Twitter for iPhone,عاجل:\n\nرسميًا:\n\nالسعودية تستضيف كأس العالم...,ar,NaN,NaN,عاجل:\n\nرسميًا:\n\nالسعودية تستضيف كأس العالم...
5106,fifaworldcup_ar,كأس العالم FIFA 🏆,2023-02-14,2784,Brand Networks Composer,من كان أفضل قلب دفاع عربي في كأس العالم للأندي...,ar,NaN,NaN,من كان أفضل قلب دفاع عربي في كأس العالم للأندي...
6758,RMCFarab,Insider Real,2023-02-13,2695,Twitter for iPhone,الفائز بدوري أبطال أوروبا\nالفائز في بالليغا\n...,ar,NaN,NaN,الفائز بدوري أبطال أوروبا\nالفائز في بالليغا\n...
4834,Ahlystudio,ستوديو الأهلي,2023-02-14,2633,Twitter for iPad,أبو تريكة في عام واحد فقط حقق كل ده .. عام ( 2...,ar,NaN,NaN,أبو تريكة في عام واحد فقط حقق كل ده .. عام ( 2...
1576,derradjihafid,hafid derradji حفيظ دراجي,2023-02-14,2296,Twitter for Android,ألف مبروك للسعودية حصولها على تنظيم كأس العالم...,ar,NaN,NaN,ألف مبروك للسعودية حصولها على تنظيم كأس العالم...
3911,hilalstuff,أخبار الهلال,2023-02-14,2262,Twitter for iPhone,🚨 في حال تحقيق الهلال لبطولة دوري أبطال آسيا ا...,ar,NaN,NaN,🚨 في حال تحقيق الهلال لبطولة دوري أبطال آسيا ا...
6880,_su6an,سلطان,2023-02-13,2086,Twitter for iPhone,فينسيوس في ٢٠٢٢\n\nسجل ٢٢ هدف - صنع ١٧ هدف - س...,ar,NaN,NaN,فينسيوس في ٢٠٢٢\n\nسجل ٢٢ هدف - صنع ١٧ هدف - س...
3229,1SMi_,سهم,2023-02-14,2040,Twitter for iPhone,🚨🚨🚨🚨— عاااااجل \n\nبطولة كأس العالم للأندية 20...,ar,NaN,NaN,🚨🚨🚨🚨— عاااااجل \n\nبطولة كأس العالم للأندية 20...


-  ( 🇸🇦 أعلى تغريدة كانت عن (السعودية تستضيف كأس العالم للأندية 2023

In [353]:
df['Total_of_Likes'] = df.groupby(['User']).Number_of_Likes.transform('sum')

In [354]:
df

,User,Name,Date_Created,Number_of_Likes,Source_of_Tweet,Tweet,Language,Coordinates,place,cleanTweet,Total_of_Likes
0,AX98jfg8k94,SAMIII,2023-02-14,0,Twitter for iPhone,@iwamzyan 😂😂😂 كأس العالم للاندية هل تطن ان الم...,ar,NaN,NaN,😂😂😂 كأس العالم للاندية هل تطن ان المغرب قام ب...,0
1,hebasay39744927,🦅♥️🇪🇬 heba.sayed‏,2023-02-14,38,Twitter for Android,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...,ar,NaN,NaN,محمد شبانة: لما أعيش لليوم اللي الزمالك يلعب ف...,40
2,2oios,-ﺎلسسَآطي ‏𖤐,2023-02-14,0,Twitter for iPhone,@toldo2002 @tooothy_ وبإذن الله ان التاسعة تتح...,ar,NaN,NaN,وبإذن الله ان التاسعة تتحق السنه ذي وكاس الع...,0
3,mohamed90108304,Mohamed Metwally Shaarawi ( ابو بـودئ )🇪🇬♥️🇸🇦,2023-02-14,1,Twitter for iPhone,"أعلن الاتحاد الدولي لكرة القدم ""#فيفا"" تطبيق ن...",ar,NaN,NaN,"أعلن الاتحاد الدولي لكرة القدم ""فيفا"" تطبيق نظ...",1
4,fstp1396,الزكرتي,2023-02-14,2,Twitter for iPhone,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...,ar,NaN,NaN,رايحين للاتحاد الاسيوي يبون ميدالية اللعب النظ...,2
...,...,...,...,...,...,...,...,...,...,...,...
9819,aljeban,أحمد | AHMED,2023-02-12,1,Twitter for iPhone,#ريال_مدريد بطل #كأس_العالم_للأندية 2022\n ...,ar,NaN,NaN,ريالمدريد بطل كأسالعالمللأندية 2022\n ...,1
9820,sayyidaty,برنامج سيدتي,2023-02-12,3,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,4
9821,YaHalaShow,برنامج ياهلا,2023-02-12,1,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,37
9822,sa3etshabab,ساعة شباب,2023-02-12,0,Twitter Media Studio - LiveCut,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,ar,NaN,NaN,ماجد المهندس: مبروك للوطن هذا الإنجاز اللي مو ...,2


In [355]:
df_sorted = df.sort_values(by='Total_of_Likes',ascending=False)
df_sorted.drop(columns=['Date_Created', 'Number_of_Likes', 'Source_of_Tweet', 'Tweet',
            'Language', 'Coordinates','place','cleanTweet'],inplace=True)
df_sorted.drop_duplicates(inplace=True)
df_sorted

,User,Name,Total_of_Likes
1527,football_ll55,نواف الآسيوي 🇸🇦,8139
646,hilalstuff,أخبار الهلال,7877
8847,fifaworldcup_ar,كأس العالم FIFA 🏆,7691
634,MnbrAlhilal,منبر الهلال,5840
4764,saudiFF,الاتحاد السعودي لكرة القدم,5756
...,...,...,...
5597,azooozyy_,عز,0
5596,wmn120,maha,0
1655,Jaber00515,ابو فيصل,0
5594,ahmd88524189,احمد القحطاني,0


In [356]:
fig =px.bar(df_sorted.head(),x='Name',y='Total_of_Likes',color='Name',
           labels={'Name':'المستخدم', 'Total_of_Likes':'عدد الاعجاب'},
           title='اعلى خمس حسابات حازت على اعجاب الاخرين')
fig.show()

In [357]:
Data=pd.DataFrame(df['Source_of_Tweet'].value_counts()).head()

In [358]:
fig = px.pie(Data, values='Source_of_Tweet', names=Data.index, title='اعلى خمس مصادر للتغريدة')
fig.show()

- اعلى مصدر للتغريدات كان عن طريق الايفون وقد نستنتج كثرت مستخدمين اجهزة الأيفون  

In [ ]:
# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(
    'يوم التاسيس until:2023-02-15 since:2023-02-01').get_items()):
    if i>100000:
        break
    attributes_container.append([tweet.user.username,tweet.user.displayname , tweet.date, 
                                 tweet.likeCount, tweet.sourceLabel, tweet.content, 
                                 tweet.lang,tweet.coordinates, tweet.place])
     
# Creating a dataframe to load the list
tweets_df = pd.DataFrame(attributes_container, columns=["User", "Name", "Date Created", 
                                                        "Number of Likes", "Source of Tweet", "Tweet", 
                                                        "Language", "Coordinates","place"])

tweets_df["cleanTweet"] = tweets_df["Tweet"].apply(lambda s: re.sub(r'(?:\@\w+)','', s))# remove string like: @kmkk 
tweets_df["cleanTweet"] = tweets_df["cleanTweet"].apply(lambda s: re.sub(r'(?:[a-zA-Z]+)','', s))# remove english word
tweets_df["cleanTweet"] = tweets_df["cleanTweet"].apply(lambda s: re.sub(  r'([$@&!~#^*+=-_]+)'    ,'', s)) 
tweets_df = tweets_df[tweets_df['Language'] ==  "ar"]
tweets_df['Date Created'] = tweets_df['Date Created'].apply(lambda a: pd.to_datetime(a).date())
print(tweets_df)
tweets_df.to_excel('TwitterData5.xlsx')

In [359]:
df = pd.read_excel('TwitterData5.xlsx')
df.columns

Index(['Unnamed: 0', 'User', 'Name', 'Date Created', 'Number of Likes',
       'Source of Tweet', 'Tweet', 'Language', 'Coordinates', 'place',
       'cleanTweet'],
      dtype='object')

In [360]:
df

,Unnamed: 0,User,Name,Date Created,Number of Likes,Source of Tweet,Tweet,Language,Coordinates,place,cleanTweet
0,0,askc3r,اسأل عن سيارتك,2023-02-14,1,Twitter for iPhone,🎈✉️من الـخـاص✉️🎈 \n\nسلام عليكم ، وش افضل الش...,ar,NaN,NaN,🎈✉️من الـخـاص✉️🎈 \n\nسلام عليكم ، وش افضل الش...
1,1,no44il,سُكرة🤍,2023-02-14,0,Twitter for iPhone,@AzizbagBag يوم التاسيس السعودي تختصر \nمشاعر ...,ar,NaN,NaN,يوم التاسيس السعودي تختصر \nمشاعر الحبّ والان...
2,2,masdarak1,عبدالرحمن,2023-02-14,0,Twitter for Android,@najeebauto الجواب \nيرمز الى أصول الدولة السع...,ar,NaN,NaN,الجواب \nيرمز الى أصول الدولة السعودية بعراقت...
3,3,MKKX_407,👑♥️.NOUFAH.👑♥️,2023-02-14,0,Twitter for iPhone,@WoC1h ٧ ملوك 🇸🇦💗\n\n.\n#مسابقة_يوم_التأسيس,ar,NaN,NaN,٧ ملوك 🇸🇦💗\n\n.\nمسابقةيومالتأسيس
4,4,Malak3234,ملاك الشهري,2023-02-14,0,Twitter for iPhone,@smco_sa #يوم_التاسيس \n\nالخدمات الإنسانية ال...,ar,NaN,NaN,يومالتاسيس \n\nالخدمات الإنسانية التي تقدمها ...
...,...,...,...,...,...,...,...,...,...,...,...
78299,92152,hilalah_1439H,•°لجدتي♡هلالة•°,2023-02-01,0,Twitter for Android,#يوم_التاسيس \nأستغفِرالله ، أستغفِرالله ، أست...,ar,NaN,NaN,يومالتاسيس \nأستغفِرالله ، أستغفِرالله ، أستغف...
78300,92153,onlyux7,Tota¹²⁷🦋🍒@ eternal sonata🌸,2023-02-01,1,Twitter for iPhone,#يوم_التاسيس لحظه يوم التأسيس قرب يعني بيكون ف...,ar,NaN,NaN,يومالتاسيس لحظه يوم التأسيس قرب يعني بيكون فيه...
78301,92154,lii0l5,دعوات الالكترونية + توزيعات,2023-02-01,0,Twitter for iPhone,#يوم_التاسيس لطلب التواصل على الانستقرام https...,ar,NaN,NaN,يومالتاسيس لطلب التواصل على الانستقرام ://./20
78302,92157,SaharEl30171576,د. سحر السيد الصايغ,2023-02-01,0,Twitter for Android,حفظكم الله ي السعودية و حفظ مصر\n#يوم_التاسيس,ar,NaN,NaN,حفظكم الله ي السعودية و حفظ مصر\nيومالتاسيس


In [361]:
df=pd.DataFrame(df)
df.drop(columns='Unnamed: 0',inplace=True)


In [362]:
df.columns=['User', 'Name', 'Date_Created', 'Number_of_Likes', 'Source_of_Tweet', 'Tweet',
            'Language', 'Coordinates','place','cleanTweet']

In [363]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78304 entries, 0 to 78303
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   User             78304 non-null  object        
 1   Name             78303 non-null  object        
 2   Date_Created     78304 non-null  datetime64[ns]
 3   Number_of_Likes  78304 non-null  int64         
 4   Source_of_Tweet  78304 non-null  object        
 5   Tweet            78304 non-null  object        
 6   Language         78304 non-null  object        
 7   Coordinates      852 non-null    object        
 8   place            852 non-null    object        
 9   cleanTweet       78304 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 6.0+ MB


In [364]:
df_sorted = df.sort_values(by='Number_of_Likes',ascending=False).head()
fig =px.bar(df_sorted,x='User',y='Number_of_Likes',hover_data=['User', 'Name','cleanTweet'],color='User',
           labels={'User':'المستخدم', 'Number_of_Likes':'عدد الاعجاب'},
           title='اعلى خمس تغريدات حازت على اعجاب الاخرين')
fig.show()

In [365]:
Data=pd.DataFrame(df['Source_of_Tweet'].value_counts()).head()

In [366]:
fig = px.pie(Data, values='Source_of_Tweet', names=Data.index, title='اعلى خمس مصادر للتغريدة')
fig.show()

- اعلى مصدر للتغريدات كان عن طريق الايفون وقد نستنتج كثرت مستخدمين اجهزة الأيفون  

In [367]:
df['Total_of_Likes'] = df.groupby(['User']).Number_of_Likes.transform('sum')

In [368]:
df_sorted = df.sort_values(by='Total_of_Likes',ascending=False)
df_sorted.drop(columns=['Date_Created', 'Number_of_Likes', 'Source_of_Tweet', 'Tweet',
            'Language', 'Coordinates','place','cleanTweet'],inplace=True)
df_sorted.drop_duplicates(inplace=True)
df_sorted

,User,Name,Total_of_Likes
50751,SAFoundingDay,يوم التأسيس,25874
75857,AzizbagBag,عبدالعزيز أحمد بغلف🇸🇦,19328
77286,News_Ejazah,وكالة تعليق الدراسة,10935
41872,uikil2,K🥛,6653
63576,lp880cqk,ابوسلطان,5280
...,...,...,...
43114,n00faksa2019,❤️ نوًوًوًوًوًوًوًف ❤️,0
43109,005315205dkh,دختووور,0
43106,1Seagull,Turki,0
43097,omsaaaalem3,رجاءكو ٢,0


In [369]:
fig =px.bar(df_sorted.head(),x='User',y='Total_of_Likes',color='User',
           labels={'User':'المستخدم', 'Total_of_Likes':'عدد الاعجاب'},
           title='اعلى خمس حسابات حازت على اعجاب الاخرين')
fig.show()

- اعلى حساب كان يتحدث عن يوم التاسيس

In [370]:
df.sort_values('Date_Created',ascending = False, inplace=True)
new=pd.DataFrame()
new['y']=df["Date_Created"].value_counts()
new['x']=df["Date_Created"].unique()
new

,y,x
2023-02-14,19242,2023-02-14
2023-02-13,16988,2023-02-13
2023-02-12,13404,2023-02-12
2023-02-04,7387,2023-02-11
2023-02-11,3610,2023-02-10
2023-02-08,2809,2023-02-09
2023-02-03,2571,2023-02-08
2023-02-07,2252,2023-02-07
2023-02-05,2107,2023-02-06
2023-02-06,2052,2023-02-05


In [371]:
fig =px.bar(x=new['x'],y=new['y'],
           labels={'x':'التاريخ', 'y':'عدد التغريدات'},
           title='عدد التغريدات لكل يوم')
fig.show()

- قد يكون السبب لقرب يوم التاسيس